Initialization

In [ ]:
import math
import json
import os
import random
from IPython import display

GREEN = "\033[92m"
RED = "\033[91m"
YELLOW = "\033[93m"
BLUE = "\033[94m"
CYAN = "\033[96m"
MAGENTA = "\033[95m"
BOLD = "\033[1m"
UNDERLINE = "\033[4m"
STANDARD = "\033[0m"

%run config.py

In [ ]:
def csv2sequence(csvFile,seqFile,pref):
	with open(csvFile, 'r') as f:
		lines = f.readlines()
		with open(seqFile, 'w') as g:
			for line in lines:
				for token in line.split(','):
					if token.startswith(pref):
						token = token[len(pref):]
					token = token.strip()
					if token!="" and token!='\n':
						g.write(token+'\n')

def sequence2csv(seqFile,csvFile,dataWidth,pref):
	with open(seqFile, 'r') as f:
		lines = f.readlines()
		dataW = dataWidth
		with open(csvFile, 'w') as g:
			for line in lines:
				g.write(pref+line.strip())
				dataW -= 1
				if dataW==0:
					g.write('\n')
					dataW = dataWidth
				else:
					g.write(',')

Create the expression using flexpytester and create inputs and outputs for the test cases.

In [ ]:
def generate():
	#! flexpytester --generate -e symbols -s generated.py > /dev/null
	! make show > /dev/null
	# TODO include the generation of the inputs and outputs
	csv2sequence('inputs.csv','inputs.seq',"0f")
	csv2sequence('outputs.csv','outputs.seq',"0f")
	# sequence2csv('inputs.seq','prova',4,"0f")

Eventually run the simbatch simulation

In [ ]:
def simbatch():
	! make simbatch > /dev/null
	#csv2sequence('simbatch_outputs.csv','simbatch_outputs.seq')

Eventually run the simulation in the FPGA with the BM

In [ ]:
def bmsim():
	print ("bmsim")

Eventually run the simulation in the FPGA with the HLS technology

In [ ]:
def hlsim():
	print ("hlsim")

Analyze the results of the chosen cases

In [ ]:
def analyze():
	print ("analyze")

Main test loop

In [ ]:
for i in range(numiterations):
	print ("Iteration: ", i)
	generate()
	simbatch()
	bmsim()
	hlsim()
	analyze()
display.Image("bondmachine.png")